In [14]:
from Dataset_HCR_ICIR_OHE_applied import *
# train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
# train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
# test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

# df_train = pd.DataFrame(train_x_raw)
# df_test = pd.DataFrame(test_x_raw)
# df_y = pd.DataFrame(train_y_raw)

# Simple Model Run
##  xgBoost model set up

In [15]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from bayes_opt import BayesianOptimization
from sklearn.metrics import log_loss

# Remap labels
label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)

## Bayesian Optimization

In [6]:
def xgb_cv_score(max_depth, gamma, colsample_bytree, subsample, eta, reg_lambda, reg_alpha, min_child_weight):
    """
    Computes the cross-validated log loss for given hyperparameter settings using Stratified K-Fold.
    """
    params = {
       # 'device': 'cuda',
        'max_depth': int(max_depth),
        'gamma': gamma,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'eta': eta,
        'objective': 'multi:softprob',
        'num_class': 5,
        'eval_metric': 'mlogloss',
        'lambda': reg_lambda,
        'alpha': reg_alpha,
        'min_child_weight': min_child_weight,
        'verbosity': 0,
        'seed': 42
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    log_loss_scores = []

    for train_index, test_index in skf.split(df_train, df_y):
        xgb_train = xgb.DMatrix(df_train.iloc[train_index], label=df_y.iloc[train_index])
        xgb_valid = xgb.DMatrix(df_train.iloc[test_index], label=df_y.iloc[test_index])
        
        watchlist = [(xgb_train, 'train'), (xgb_valid, 'eval')]

        # Add early_stopping_rounds
        model = xgb.train(params, xgb_train, num_boost_round=500, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)

        # Predict using the best iteration
        preds = model.predict(xgb_valid)
        #preds = model.predict(xgb_valid, ntree_limit=(model.best_iteration + 1) * params['num_class']

        log_loss_score = log_loss(df_y.iloc[test_index], preds, labels=list(range(5)))
        log_loss_scores.append(log_loss_score)

    return -np.mean(log_loss_scores)

# Define the hyperparameter bounds
pbounds = {
    'max_depth': (3, 7),  # Reduced upper limit to prevent overfitting and reduce complexity.
    'gamma': (0.1, 4),  # Expanded to include lower values, allowing for less aggressive regularization at the low end.
    'colsample_bytree': (0.5, 0.8),  # Narrowed to focus on moderately high feature sampling for balance between performance and overfitting.
    'subsample': (0.5, 0.8),  # Similar to colsample_bytree, aiming for a balance in sampling.
    'eta': (0.01, 0.2),  # Narrowed upper limit to focus on more stable, albeit potentially slower, learning rates.
    'reg_lambda': (0.5, 4),  # Expanded lower bound to allow for less regularization, adjusted upper limit to prevent overly strong L2 regularization.
    'reg_alpha': (0, 0.5),  # Reduced upper limit to focus on lower levels of L1 regularization, preventing sparsity from being too aggressive.
    'min_child_weight': (1, 5),  # Narrowed to prevent overly conservative model by avoiding too large values.
}

# Perform Bayesian Optimization
optimizer = BayesianOptimization(f=xgb_cv_score, pbounds=pbounds, random_state=42, verbose=2)
optimizer.maximize(init_points=10, n_iter=30)

|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
| 1         | -0.8494   | 0.6124    | 0.1906    | 2.955     | 5.395     | 1.624     | 0.078     | 0.7033    | 0.7599    |
| 2         | -0.8496   | 0.6803    | 0.1445    | 0.1803    | 6.88      | 4.33      | 0.1062    | 1.136     | 0.555     |
| 3         | -0.8468   | 0.5913    | 0.1097    | 1.785     | 4.165     | 3.447     | 0.06975   | 1.523     | 0.6099    |
| 4         | -0.8501   | 0.6368    | 0.1592    | 0.8787    | 5.057     | 3.37      | 0.02323   | 2.626     | 0.5512    |
| 5         | -0.8494   | 0.5195    | 0.1903    | 3.866     | 6.234     | 2.218     | 0.04884   | 2.895     | 0.632     |
| 6         | -0.842    | 0.5366    | 0.1041    | 0.2341    | 6.637     | 2.035     | 0.3313    | 1.591     | 0.656     |
| 7         | -0.8439   

## CV xgboost train with best parameters

In [18]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size=0.3, random_state=42)

# Best parameters from optimization
best_params = {
    'max_depth': int(optimizer.max['params']['max_depth']),
    'gamma': 3,
    'colsample_bytree': optimizer.max['params']['colsample_bytree'],
    'subsample': optimizer.max['params']['subsample'],
    'eta': optimizer.max['params']['eta'],
    'lambda': optimizer.max['params']['reg_lambda'],
    'alpha': optimizer.max['params']['reg_alpha'],
    'min_child_weight': optimizer.max['params']['min_child_weight'],
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'mlogloss',
    'verbosity': 0,
    'seed': 42
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [17]:
evals_result = {}
bst = xgb.train(best_params, dtrain, num_boost_round=1000, evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10, evals_result=evals_result, verbose_eval=True)

# Evaluate and print the final training and validation loss
train_last_eval = evals_result['train']['mlogloss'][-1]
val_last_eval = evals_result['val']['mlogloss'][-1]

print(f"Training Multiclass Logarithmic Loss: {train_last_eval}")
print(f"Validation Multiclass Logarithmic Loss: {val_last_eval}")

[0]	train-mlogloss:1.57664	val-mlogloss:1.57923
[1]	train-mlogloss:1.54505	val-mlogloss:1.54995
[2]	train-mlogloss:1.51408	val-mlogloss:1.52172
[3]	train-mlogloss:1.48480	val-mlogloss:1.49504
[4]	train-mlogloss:1.45733	val-mlogloss:1.46995
[5]	train-mlogloss:1.42983	val-mlogloss:1.44519
[6]	train-mlogloss:1.40611	val-mlogloss:1.42421
[7]	train-mlogloss:1.38241	val-mlogloss:1.40274
[8]	train-mlogloss:1.36141	val-mlogloss:1.38380
[9]	train-mlogloss:1.34024	val-mlogloss:1.36473
[10]	train-mlogloss:1.32048	val-mlogloss:1.34721
[11]	train-mlogloss:1.30185	val-mlogloss:1.33061
[12]	train-mlogloss:1.28266	val-mlogloss:1.31376
[13]	train-mlogloss:1.26463	val-mlogloss:1.29805
[14]	train-mlogloss:1.24835	val-mlogloss:1.28376
[15]	train-mlogloss:1.23273	val-mlogloss:1.27022
[16]	train-mlogloss:1.21609	val-mlogloss:1.25591
[17]	train-mlogloss:1.20211	val-mlogloss:1.24367
[18]	train-mlogloss:1.18921	val-mlogloss:1.23272
[19]	train-mlogloss:1.17546	val-mlogloss:1.22091
[20]	train-mlogloss:1.16154	va

KeyboardInterrupt: 

## Generate Submission csv

In [ ]:
# Combine X_train and X_val, and y_train and y_val
X_full = pd.concat([X_train, X_val], axis=0)
y_full = pd.concat([y_train, y_val], axis=0)

dfull = xgb.DMatrix(X_full, label=y_full)

# Retrain the model
evals_result_full = {}
bst_full = xgb.train(best_params, dfull, num_boost_round=1000, evals=[(dfull, 'full')],
                     early_stopping_rounds=10, evals_result=evals_result_full, verbose_eval=True)

train_full_last_eval = evals_result_full['full']['mlogloss'][-1]
print(f"Full Training Multiclass Logarithmic Loss: {train_full_last_eval}")

[0]	full-mlogloss:1.57695
[1]	full-mlogloss:1.54542
[2]	full-mlogloss:1.51453
[3]	full-mlogloss:1.48548
[4]	full-mlogloss:1.45824
[5]	full-mlogloss:1.43111
[6]	full-mlogloss:1.40788
[7]	full-mlogloss:1.38428
[8]	full-mlogloss:1.36341
[9]	full-mlogloss:1.34247
[10]	full-mlogloss:1.32303
[11]	full-mlogloss:1.30470
[12]	full-mlogloss:1.28606
[13]	full-mlogloss:1.26843
[14]	full-mlogloss:1.25261
[15]	full-mlogloss:1.23731
[16]	full-mlogloss:1.22121
[17]	full-mlogloss:1.20753
[18]	full-mlogloss:1.19477
[19]	full-mlogloss:1.18131
[20]	full-mlogloss:1.16787
[21]	full-mlogloss:1.15534
[22]	full-mlogloss:1.14366
[23]	full-mlogloss:1.13168
[24]	full-mlogloss:1.12100
[25]	full-mlogloss:1.10952
[26]	full-mlogloss:1.09881
[27]	full-mlogloss:1.08881
[28]	full-mlogloss:1.07847
[29]	full-mlogloss:1.06910
[30]	full-mlogloss:1.05987
[31]	full-mlogloss:1.04993
[32]	full-mlogloss:1.04138
[33]	full-mlogloss:1.03302
[34]	full-mlogloss:1.02526
[35]	full-mlogloss:1.01700
[36]	full-mlogloss:1.00962
[37]	full-m

In [13]:
dtest = xgb.DMatrix(df_test)
y_test_probs = bst_full.predict(dtest)
class_order = [0, 1, 2, 3, 4]
class_mapping = {class_label: f"Class{class_label}" for class_label in class_order}

submission_df = pd.DataFrame(y_test_probs, columns=class_mapping.values())
submission_df.columns = ['no answer', 'very important', 'quite important', 'not important', 'not at all important']
submission_df.insert(0, 'id', df_test.index)

# Save the submission file
submission_file = ('eda_script_og_amy_submission.csv')
submission_df.to_csv(submission_file, index=False)